# 3   Processing Raw Text

The most important source of texts is undoubtedly the Web. It's convenient to have existing text collections to explore, such as the corpora we saw in the previous chapters. However, you probably have your own text sources in mind, and need to learn how to access them.

The goal of this chapter is to answer the following questions:

- How can we write programs to access text from local files and from the web, in order to get hold of an unlimited range of language material?
- How can we split documents up into individual words and punctuation symbols, so we can carry out the same kinds of analysis we did with text corpora in earlier chapters?
- How can we write programs to produce formatted output and save it in a file?

In order to address these questions, we will be covering key concepts in NLP, including tokenization and stemming. Along the way you will consolidate your Python knowledge and learn about strings, files, and regular expressions. Since so much text on the web is in HTML format, we will also see how to dispense with markup.

In [6]:
import nltk, re, pprint
from nltk import word_tokenize

## 3.1   Accessing Text from the Web and from Disk

### Electronic Books

A small sample of texts from Project Gutenberg appears in the NLTK corpus collection. However, you may be interested in analyzing other texts from Project Gutenberg. You can browse the catalog of 25,000 free online books at http://www.gutenberg.org/catalog/, and obtain a URL to an ASCII text file. Although 90% of the texts in Project Gutenberg are in English, it includes material in over 50 other languages, including Catalan, Chinese, Dutch, Finnish, French, German, Italian, Portuguese and Spanish (with more than 100 texts each).

Text number 2554 is an English translation of *Crime and Punishment*, and we can access it as follows.

In [2]:
from urllib import request
url = "http://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')
type(raw)

str

In [3]:
len(raw)

1176812

In [4]:
raw[:76]

'\ufeffThe Project Gutenberg eBook of Crime and Punishment, by Fyodor Dostoevsky\r\n'

The variable `raw` contains a string with 1,176,967 characters. (We can see that it is a string, using `type(raw)`.) This is the raw content of the book, including many details we are not interested in such as whitespace, line breaks and blank lines. Notice the `\r` and `\n` in the opening line of the file, which is how Python displays the special carriage return and line feed characters (the file must have been created on a Windows machine). For our language processing, we want to break up the string into words and punctuation, as we saw in Chapter 1. This step is called **tokenization**, and it produces our familiar structure, a list of words and punctuation.

In [5]:
tokens = word_tokenize(raw)
type(tokens)

list

In [ ]:
len(tokens)

In [6]:
tokens[:10]

['\ufeffThe',
 'Project',
 'Gutenberg',
 'eBook',
 'of',
 'Crime',
 'and',
 'Punishment',
 ',',
 'by']

Notice that NLTK was needed for tokenization, but not for any of the earlier tasks of opening a URL and reading it into a string. If we now take the further step of creating an NLTK text from this list, we can carry out all of the other linguistic processing we saw in Chapter 1, along with the regular list operations like slicing:

In [7]:
text = nltk.Text(tokens)
type(text)

nltk.text.Text

In [8]:
text[1025:1063]

['impresses',
 'us',
 'as',
 'wisdom',
 '...',
 'that',
 'wisdom',
 'of',
 'the',
 'heart',
 'which',
 'we',
 'seek',
 'that',
 'we',
 'may',
 'learn',
 'from',
 'it',
 'how',
 'to',
 'live',
 '.',
 'All',
 'his',
 'other',
 'gifts',
 'came',
 'to',
 'him',
 'from',
 'nature',
 ',',
 'this',
 'he',
 'won',
 'for',
 'himself']

In [9]:
text.collocations()

Katerina Ivanovna; Pyotr Petrovitch; Pulcheria Alexandrovna; Avdotya
Romanovna; Rodion Romanovitch; Marfa Petrovna; Sofya Semyonovna; old
woman; Project Gutenberg-tm; Porfiry Petrovitch; Amalia Ivanovna;
great deal; young man; Nikodim Fomitch; Project Gutenberg; Ilya
Petrovitch; Andrey Semyonovitch; Hay Market; Dmitri Prokofitch; Good
heavens


Notice that *Project Gutenberg* appears as a collocation. This is because each text downloaded from Project Gutenberg contains a header with the name of the text, the author, the names of people who scanned and corrected the text, a license, and so on. Sometimes this information appears in a footer at the end of the file. We cannot reliably detect where the content begins and ends, and so have to resort to manual inspection of the file, to discover unique strings that mark the beginning and the end, before trimming `raw` to be just the content and nothing else:

In [10]:
raw.find("PART I")

5575

In [11]:
raw.rfind("End of Project Gutenberg’s Crime")

-1

In [12]:
raw[1157812:1157844]

'adual\r\nrenewal of a man, the sto'

In [13]:
raw = raw[5336:1157812]

In [14]:
raw.find("PART I")

239

The `find()` and `rfind()` ("reverse find") methods help us get the right index values to use for slicing the string. We overwrite `raw` with this slice, so now it begins with "PART I" and goes up to (but not including) the phrase that marks the end of the content.

This was our first brush with the reality of the web: texts found on the web may contain unwanted material, and there may not be an automatic way to remove it. But with a small amount of extra work we can extract the material we need.

### Dealing with HTML

Much of the text on the web is in the form of HTML documents. You can use a web browser to save a page as text to a local file, then access this as described in the section on files below. However, if you're going to do this often, it's easiest to get Python to do the work directly. The first step is the same as before, using `urlopen`. For fun we'll pick a BBC News story called *Blondes to die out in 200 years*, an urban legend passed along by the BBC as established scientific fact:

In [15]:
url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"
html = request.urlopen(url).read().decode('utf8')
html[:60]

'<!doctype html public "-//W3C//DTD HTML 4.0 Transitional//EN'

You can type `print(html)` to see the HTML content in all its glory, including meta tags, an image map, JavaScript, forms, and tables.

To get text out of HTML we will use a Python library called *BeautifulSoup*, available from http://www.crummy.com/software/BeautifulSoup/:

In [16]:
from bs4 import BeautifulSoup
raw = BeautifulSoup(html, 'html.parser').get_text()
tokens = word_tokenize(raw)
tokens

['BBC',
 'NEWS',
 '|',
 'Health',
 '|',
 'Blondes',
 "'to",
 'die',
 'out',
 'in',
 '200',
 "years'",
 'NEWS',
 'SPORT',
 'WEATHER',
 'WORLD',
 'SERVICE',
 'A-Z',
 'INDEX',
 'SEARCH',
 'You',
 'are',
 'in',
 ':',
 'Health',
 'News',
 'Front',
 'Page',
 'Africa',
 'Americas',
 'Asia-Pacific',
 'Europe',
 'Middle',
 'East',
 'South',
 'Asia',
 'UK',
 'Business',
 'Entertainment',
 'Science/Nature',
 'Technology',
 'Health',
 'Medical',
 'notes',
 '--',
 '--',
 '--',
 '--',
 '--',
 '--',
 '-',
 'Talking',
 'Point',
 '--',
 '--',
 '--',
 '--',
 '--',
 '--',
 '-',
 'Country',
 'Profiles',
 'In',
 'Depth',
 '--',
 '--',
 '--',
 '--',
 '--',
 '--',
 '-',
 'Programmes',
 '--',
 '--',
 '--',
 '--',
 '--',
 '--',
 '-',
 'SERVICES',
 'Daily',
 'E-mail',
 'News',
 'Ticker',
 'Mobile/PDAs',
 '--',
 '--',
 '--',
 '--',
 '--',
 '--',
 '-',
 'Text',
 'Only',
 'Feedback',
 'Help',
 'EDITIONS',
 'Change',
 'to',
 'UK',
 'Friday',
 ',',
 '27',
 'September',
 ',',
 '2002',
 ',',
 '11:51',
 'GMT',
 '12:51'

This still contains unwanted material concerning site navigation and related stories. With some trial and error you can find the start and end indexes of the content and select the tokens of interest, and initialize a text as before.

In [17]:
tokens = tokens[110:390]
text = nltk.Text(tokens)
text.concordance('gene')

Displaying 5 of 5 matches:
hey say too few people now carry the gene for blondes to last beyond the next 
blonde hair is caused by a recessive gene . In order for a child to have blond
 have blonde hair , it must have the gene on both sides of the family in the g
ere is a disadvantage of having that gene or by chance . They do n't disappear
des would disappear is if having the gene was a disadvantage and I do not thin


### Processing Search Engine Results

The web can be thought of as a huge corpus of unannotated text. Web search engines provide an efficient means of searching this large quantity of text for relevant linguistic examples. The main advantage of search engines is size: since you are searching such a large set of documents, you are more likely to find any linguistic pattern you are interested in. Furthermore, you can make use of very specific patterns, which would only match one or two examples on a smaller corpus, but which might match tens of thousands of examples when run on the web. A second advantage of web search engines is that they are very easy to use. Thus, they provide a very convenient tool for quickly checking a theory, to see if it is reasonable.

*Google Hits for Collocations: The number of hits for collocations involving the words absolutely or definitely, followed by one of adore, love, like, or prefer.*

|Google hits 	|*adore* 	|*love* 	|*like* 	|*prefer*|
|:-|-:|-:|-:|-:|
|*absolutely* 	|289,000 	|905,000 	|16,200 	|644|
|*definitely* 	|1,460 	|51,000 	|158,000 	|62,600|
|ratio 	|198:1 	|18:1 	|1:10 	|1:97|

Unfortunately, search engines have some significant shortcomings. First, the allowable range of search patterns is severely restricted. Unlike local corpora, where you write programs to search for arbitrarily complex patterns, search engines generally only allow you to search for individual words or strings of words, sometimes with wildcards. Second, search engines give inconsistent results, and can give widely different figures when used at different times or in different geographical regions. When content has been duplicated across multiple sites, search results may be boosted. Finally, the markup in the result returned by a search engine may change unpredictably, breaking any pattern-based method of locating particular content (a problem which is ameliorated by the use of search engine APIs).

### Processing RSS Feeds

The blogosphere is an important source of text, in both formal and informal registers. With the help of a Python library called the *Universal Feed Parser*, available from https://pypi.python.org/pypi/feedparser, we can access the content of a blog, as shown below:

In [18]:
import feedparser
llog = feedparser.parse("http://languagelog.ldc.upenn.edu/nll/?feed=atom")
llog['feed']['title']

'Language Log'

In [19]:
len(llog.entries)

13

In [20]:
post = llog.entries[2]
post.title

'Speaking Taiwanese as a Second Language in Taiwan'

In [21]:
content = post.content[0].value
content[:70]

'<p>Provocative Twitter thread:</p>\n<p><blockquote class="twitter-tweet'

In [22]:
raw = BeautifulSoup(content, 'html.parser').get_text()
word_tokenize(raw)

['Provocative',
 'Twitter',
 'thread',
 ':',
 'I',
 'wanted',
 'to',
 'write',
 'a',
 'thread',
 'documenting',
 'my',
 'experiences',
 'as',
 'a',
 'millennial',
 'Taiwanese-American',
 'woman',
 'speaking',
 'Taiwanese',
 '台語',
 'in',
 'northern',
 'Taiwan',
 '–',
 'outside',
 'the',
 'family',
 ',',
 'with',
 'a',
 'foreign',
 'accent',
 '–',
 'and',
 'why',
 'I',
 'am',
 'adamant',
 'the',
 'ROC',
 'is',
 'a',
 'colonial',
 'state',
 'for',
 'robbing',
 'people',
 'of',
 'their',
 'mother',
 'languages',
 '1/—',
 'Catie',
 'Lilly',
 '(',
 '@',
 'catielila',
 ')',
 'September',
 '28',
 ',',
 '2021',
 'The',
 'lengthy',
 'thread',
 'goes',
 'on',
 'to',
 'describe',
 'various',
 'people',
 "'s",
 'reactions',
 'to',
 'her',
 'speaking',
 'Taiwanese',
 'to',
 'them',
 'as',
 'well',
 'as',
 'the',
 'writer',
 "'s",
 'own',
 'thoughts',
 '.',
 'It',
 'takes',
 'into',
 'account',
 'the',
 'cultural',
 ',',
 'economic',
 ',',
 'political',
 ',',
 'and',
 'linguistic',
 'dimensions',
 'o

With some further work, we can write programs to create a small corpus of blog posts, and use this as the basis for our NLP work.

### Reading Local Files

In order to read a local file, we need to use Python's built-in `open()` function, followed by the `read()` method. Suppose you have a file `document.txt`, you can load its contents like this:

In [ ]:
f = open('document.txt')
raw = f.read()
raw

The `read()` method creates a string with the contents of the entire file. Recall that the `'\n'` characters are **newlines**; this is equivalent to pressing *Enter* on a keyboard and starting a new line.

We can also read a file one line at a time using a `for` loop:

In [ ]:
f = open('document.txt')
for line in f:
    print(line.strip())

Here we use the `strip()` method to remove the newline character at the end of the input line.

NLTK's corpus files can also be accessed using these methods. We simply have to use `nltk.data.find()` to get the filename for any corpus item. Then we can open and read it in the way we just demonstrated above:

In [ ]:
path = nltk.data.find('corpora/gutenberg/melville-moby_dick.txt')
open(path).read()

### Capturing User Input

Sometimes we want to capture the text that a user inputs when she is interacting with our program. To prompt the user to type a line of input, call the Python function `input()`. After saving the input to a variable, we can manipulate it just as we have done for other strings.
  	

In [ ]:
#s = input("Enter some text: ")
#print("You typed", len(word_tokenize(s)), "words.")

### The NLP Pipeline

The following figure summarizes what we have covered in this section, including the process of building a vocabulary that we saw in Chapter 1. (One step, normalization, will be discussed in Section 3.6.)

<img src='images/pipeline1.png' width='600'/>

*The Processing Pipeline: We open a URL and read its HTML content, remove the markup and select a slice of characters; this is then tokenized and optionally converted into an `nltk.Text` object; we can also lowercase all the words and extract the vocabulary.*

There's a lot going on in this pipeline. To understand it properly, it helps to be clear about the type of each variable that it mentions. We find out the type of any Python object `x` using `type(x)`, e.g. `type(1)` is `<int>` since `1` is an integer.

When we load the contents of a URL or file, and when we strip out HTML markup, we are dealing with strings, Python's `<str>` data type. (We will learn more about strings in Section 3.2):

In [ ]:
raw = open('document.txt').read()
type(raw)

When we tokenize a string we produce a list (of words), and this is Python's `<list>` type. Normalizing and sorting lists produces other lists:

In [ ]:
tokens = word_tokenize(raw)
type(tokens)

In [ ]:
words = [w.lower() for w in tokens]
type(words)

In [ ]:
vocab = sorted(set(words))
type(vocab)

The type of an object determines what operations you can perform on it. So, for example, we can append to a list but not to a string:

In [ ]:
vocab.append('blog')
#raw.append('blog')

Similarly, we can concatenate strings with strings, and lists with lists, but we cannot concatenate strings with lists:

In [ ]:
query = 'Who knows?'
beatles = ['john', 'paul', 'george', 'ringo']
#query + beatles

## 3.2   Strings: Text Processing at the Lowest Level

It's time to examine a fundamental data type that we've been studiously avoiding so far. In earlier chapters we focused on a text as a list of words. We didn't look too closely at words and how they are handled in the programming language. By using NLTK's corpus interface we were able to ignore the files that these texts had come from. The contents of a word, and of a file, are represented by programming languages as a fundamental data type known as a **string**. In this section we explore strings in detail, and show the connection between strings, words, texts and files.

### Basic Operations with Strings

Strings are specified using single quotes or double quotes, as shown below. If a string contains a single quote, we must backslash-escape the quote so Python knows a literal quote character is intended, or else put the string in double quotes. Otherwise, the quote inside the string will be interpreted as a close quote, and the Python interpreter will report a syntax error:

In [30]:
monty = 'Monty Python'
monty

'Monty Python'

In [ ]:
circus = "Monty Python's Flying Circus"
circus

In [ ]:
circus = 'Monty Python\'s Flying Circus'
circus

In [ ]:
#circus = 'Monty Python's Flying Circus'

Sometimes strings go over several lines. Python provides us with various ways of entering them. In the next example, a sequence of two strings is joined into a single string. We need to use backslash or parentheses so that the interpreter knows that the statement is not complete after the first line.  	

In [23]:
couplet = "Shall I compare thee to a Summer's day?"\
            "Thou are more lovely and more temperate:"
print(couplet)

Shall I compare thee to a Summer's day?Thou are more lovely and more temperate:


In [24]:
couplet = ("Rough winds do shake the darling buds of May,"
           "And Summer's lease hath all too short a date:")
print(couplet)

Rough winds do shake the darling buds of May,And Summer's lease hath all too short a date:


Unfortunately the above methods do not give us a newline between the two lines of the sonnet. Instead, we can use a triple-quoted string as follows:  	

In [25]:
couplet = """Shall I compare thee to a Summer's day?
Thou are more lovely and more temperate:"""
print(couplet)

Shall I compare thee to a Summer's day?
Thou are more lovely and more temperate:


In [26]:
couplet = '''Rough winds do shake the darling buds of May,
And Summer's lease hath all too short a date:'''
print(couplet)

Rough winds do shake the darling buds of May,
And Summer's lease hath all too short a date:


Now that we can define strings, we can try some simple operations on them. First let's look at the `+` operation, known as **concatenation**. It produces a new string that is a copy of the two original strings pasted together end-to-end. Notice that concatenation doesn't do anything clever like insert a space between the words. We can even multiply strings:

In [27]:
'very' + 'very' + 'very'

'veryveryvery'

In [28]:
'very' * 3

'veryveryvery'

We've seen that the addition and multiplication operations apply to strings, not just numbers. However, note that we cannot use subtraction or division with strings:  	

In [ ]:
#'very' - 'y'

In [ ]:
#'very' / 2

These error messages are another example of Python telling us that we have got our data types in a muddle. In the first case, we are told that the operation of subtraction (i.e., `-`) cannot apply to objects of type `str` (strings), while in the second, we are told that division cannot take `str` and `int` as its two operands.

### Printing Strings

So far, when we have wanted to look at the contents of a variable or see the result of a calculation, we have just typed the variable name into the interpreter. We can also see the contents of a variable using the `print` statement: 

In [ ]:
print(monty)

Notice that there are no quotation marks this time. When we inspect a variable by typing its name in the interpreter, the interpreter prints the Python representation of its value. Since it's a string, the result is quoted. However, when we tell the interpreter to `print` the contents of the variable, we don't see quotation characters since there are none inside the string.

The `print` statement allows us to display more than one item on a line in various ways, as shown below:

In [ ]:
grail = 'Holy Grail'
print(monty + grail)
print(monty, grail)
print(monty, "and the", grail)

### Accessing Individual Characters

As we saw in Chapter 1 for lists, strings are indexed, starting from zero. When we index a string, we get one of its characters (or letters). A single character is nothing special — it's just a string of length `1`.

In [ ]:
monty[0]

In [ ]:
monty[3]

In [ ]:
monty[5]

As with lists, if we try to access an index that is outside of the string we get an error:

In [ ]:
#monty[20]

Again as with lists, we can use negative indexes for strings, where `-1` is the index of the last character. Positive and negative indexes give us two ways to refer to any position in a string. In this case, when the string had a length of 12, indexes `5` and `-7` both refer to the same character (a space). (Notice that `5 = len(monty) - 7`.)

In [33]:
monty[-1]

'Monty Python'

In [34]:
monty[5]

' '

In [35]:
monty[-7]

' '

We can write `for` loops to iterate over the characters in strings. This `print` function includes the optional `end=' '` parameter, which is how we tell Python to print a space instead of a newline at the end.

In [36]:
sent = 'colorless green ideas sleep furiously'
for char in sent:
    print(char, end=' ')

c o l o r l e s s   g r e e n   i d e a s   s l e e p   f u r i o u s l y 

We can count individual characters as well. We should ignore the case distinction by normalizing everything to lowercase, and filter out non-alphabetic characters:

In [37]:
from nltk.corpus import gutenberg
raw = gutenberg.raw('melville-moby_dick.txt')
fdist = nltk.FreqDist(ch.lower() for ch in raw if ch.isalpha())
fdist.most_common(5)

[('e', 117092), ('t', 87996), ('a', 77916), ('o', 69326), ('n', 65617)]

In [ ]:
[char for (char, count) in fdist.most_common()]

This gives us the letters of the alphabet, with the most frequently occurring letters listed first (this is quite complicated and we'll explain it more carefully below). You might like to visualize the distribution using `fdist.plot()`. The relative character frequencies of a text can be used in automatically identifying the language of the text.

### Accessing Substrings

<img src='images/string-slicing.png' width='400' />

*String Slicing: The string "Monty Python" is shown along with its positive and negative indexes; two substrings are selected using "slice" notation. The slice `[m,n]` contains the characters from position `m` through `n-1`.*

A substring is any continuous section of a string that we want to pull out for further processing. We can easily access substrings using the same slice notation we used for lists. For example, the following code accesses the substring starting at index `6`, up to (but not including) index `10`:

In [ ]:
monty[6:10]

Here we see the characters are `'P'`, `'y'`, `'t'`, and `'h'` which correspond to `monty[6] ... monty[9]` but not `monty[10]`. This is because a slice *starts* at the first index but finishes *one before* the end index.

We can also slice with negative indexes — the same basic rule of starting from the start index and stopping one before the end index applies; here we stop before the space character.

In [16]:
monty[-12:-7]

NameError: name 'monty' is not defined

As with list slices, if we omit the first value, the substring begins at the start of the string. If we omit the second value, the substring continues to the end of the string:

In [ ]:
monty[:5]

In [ ]:
monty[6:]

We test if a string contains a particular substring using the `in` operator, as follows:

In [ ]:
phrase = 'And now for something completely different'
if 'thing' in phrase:
    print('found "thing"')

We can also find the position of a substring within a string, using `find()`:

In [ ]:
monty.find('Python')

### More operations on strings

Python has comprehensive support for processing strings. A summary, including some operations we haven't seen yet, is shown below. For more information on strings, type `help(str)` at the Python prompt.

*Useful String Methods: operations on strings in addition to the string tests shown in Chapter 1; all methods produce a new string or list*

|Method| 	Functionality|
|:-|:-|
|`s.find(t)`| 	index of first instance of string `t` inside `s` (`-1` if not found)|
|`s.rfind(t)`| 	index of last instance of string `t` inside `s` (`-1` if not found)|
|`s.index(t)`| 	like `s.find(t)` except it raises `ValueError` if not found|
|`s.rindex(t)`| 	like `s.rfind(t)` except it raises `ValueError` if not found|
|`s.join(text)`| 	combine the words of the text into a string using `s` as the glue|
|`s.split(t)`| 	split `s` into a list wherever a `t` is found (whitespace by default)|
|`s.splitlines()`| 	split `s` into a list of strings, one per line|
|`s.lower()`| 	a lowercased version of the string `s`|
|`s.upper()`| 	an uppercased version of the string `s`|
|`s.title()`| 	a titlecased version of the string `s`|
|`s.strip()`| 	a copy of `s` without leading or trailing whitespace|
|`s.replace(t, u)`| 	replace instances of `t` with `u` inside `s`|

### The Difference between Lists and Strings

Strings and lists are both kinds of **sequence**. We can pull them apart by indexing and slicing them, and we can join them together by concatenating them. However, we cannot join strings and lists:

In [ ]:
query = 'Who knows?'
beatles = ['John', 'Paul', 'George', 'Ringo']
query[2]

In [ ]:
beatles[2]

In [ ]:
query[:2]

In [ ]:
beatles[:2]

In [ ]:
query + " I don't"

In [ ]:
#beatles + 'Brian'

In [ ]:
beatles + ['Brian']

When we open a file for reading into a Python program, we get a string corresponding to the contents of the whole file. If we use a `for` loop to process the elements of this string, all we can pick out are the individual characters — we don't get to choose the granularity. By contrast, the elements of a list can be as big or small as we like: for example, they could be paragraphs, sentences, phrases, words, characters. So lists have the advantage that we can be flexible about the elements they contain, and correspondingly flexible about any downstream processing. Consequently, one of the first things we are likely to do in a piece of NLP code is tokenize a string into a list of strings (see Section 3.7). Conversely, when we want to write our results to a file, or to a terminal, we will usually format them as a string (see Section 3.9).

Lists and strings do not have exactly the same functionality. Lists have the added power that you can change their elements:

In [ ]:
beatles[0] = "John Lennon"
del beatles[-1]
beatles

On the other hand if we try to do that with a *string* — changing the 0th character in `query` to `'F'` — we get:

In [ ]:
#query[0] = 'F'

This is because strings are **immutable** — you can't change a string once you have created it. However, lists are **mutable**, and their contents can be modified at any time. As a result, lists support operations that modify the original value rather than producing a new value.

## 3.3   Text Processing with Unicode

Our programs will often need to deal with different languages, and different character sets. The concept of "plain text" is a fiction. If you live in the English-speaking world you probably use ASCII, possibly without realizing it. If you live in Europe you might use one of the extended Latin character sets, containing such characters as "ø" for Danish and Norwegian, "ő" for Hungarian, "ñ" for Spanish and Breton, and "ň" for Czech and Slovak. In this section, we will give an overview of how to use Unicode for processing texts that use non-ASCII character sets.

### What is Unicode?

Unicode supports over a million characters. Each character is assigned a number, called a **code point**. In Python, code points are written in the form \uXXXX, where XXXX is the number in 4-digit hexadecimal form.

Within a program, we can manipulate Unicode strings just like normal strings. However, when Unicode characters are stored in files or displayed on a terminal, they must be encoded as a stream of bytes. Some encodings (such as ASCII and Latin-2) use a single byte per code point, so they can only support a small subset of Unicode, enough for a single language. Other encodings (such as UTF-8) use multiple bytes and can represent the full range of Unicode characters.

Text in files will be in a particular encoding, so we need some mechanism for translating it into Unicode — translation into Unicode is called **decoding**. Conversely, to write out Unicode to a file or a terminal, we first need to translate it into a suitable **encoding** — this translation out of Unicode is called encoding, and is illustrated below.

<img src='images/unicode.png' width='400' />

*Unicode Decoding and Encoding*

From a Unicode perspective, characters are abstract entities which can be realized as one or more **glyphs**. Only glyphs can appear on a screen or be printed on paper. A font is a mapping from characters to glyphs.

### Extracting encoded text from files

Let's assume that we have a small text file, and that we know how it is encoded. For example, `polish-lat2.txt`, as the name suggests, is a snippet of Polish text (from the Polish Wikipedia; see http://pl.wikipedia.org/wiki/Biblioteka_Pruska). This file is encoded as Latin-2, also known as ISO-8859-2. The function `nltk.data.find()` locates the file for us.

In [ ]:
path = nltk.data.find('corpora/unicode_samples/polish-lat2.txt')

The Python `open()` function can read encoded data into Unicode strings, and write out Unicode strings in encoded form. It takes a parameter to specify the encoding of the file being read or written. So let's open our Polish file with the encoding `'latin2'` and inspect the contents of the file:

In [ ]:
f = open(path, encoding='latin2')
for line in f:
   line = line.strip()
   print(line)

If we want to see the underlying numerical values (or "codepoints") of the characters, then we can convert all non-ASCII characters into their two-digit \xXX and four-digit \uXXXX representations:

In [ ]:
f = open(path, encoding='latin2')
for line in f:
    line = line.strip()
    print(line.encode('unicode_escape'))

The first line above illustrates a Unicode escape string preceded by the `\u` escape string, namely `\u0144`. The relevant Unicode character will be displayed on the screen as the glyph ń. In the third line, we see `\xf3`, which corresponds to the glyph ó, and is within the 128-255 range.

In Python 3, source code is encoded using UTF-8 by default, and you can include Unicode characters in strings if you are using a program editor that supports Unicode. Arbitrary Unicode characters can be included using the \uXXXX escape sequence. We find the integer ordinal of a character using `ord()`. For example:

In [1]:
ord('ń')

324

The hexadecimal 4 digit notation for 324 is 0144 (type `hex(324)` to discover this), and we can define a string with the appropriate escape sequence.

In [2]:
nacute = '\u0144'
nacute

'ń'

We can also see how this character is represented as a sequence of bytes inside a text file:

In [3]:
nacute.encode('utf8')

b'\xc5\x84'

The module `unicodedata` lets us inspect the properties of Unicode characters. In the following example, we select all characters in the third line of our Polish text outside the ASCII range and print their UTF-8 byte sequence, followed by their code point integer using the standard Unicode convention (i.e., prefixing the hex digits with `U+`), followed by their Unicode name.

In [4]:
import unicodedata
lines = open(path, encoding='latin2').readlines()
line = lines[2]
print(line.encode('unicode_escape'))

NameError: name 'path' is not defined

In [ ]:
for c in line: 
    if ord(c) > 127:
        print('{} U+{:04x} {}'.format(c.encode('utf8'), ord(c), unicodedata.name(c)))

If you replace `c.encode('utf8')` with `c`, you should see an output like the following:

In [ ]:
for c in line: 
    if ord(c) > 127:
        print('{} U+{:04x} {}'.format(c, ord(c), unicodedata.name(c)))

The next examples illustrate how Python string methods and the `re` module can work with Unicode characters. (We will take a close look at the `re` module in the following section. The `\w` matches a "word character").

In [ ]:
line.find('zosta\u0142y')

In [ ]:
line = line.lower()
line

In [ ]:
line.encode('unicode_escape')

In [ ]:
import re
m = re.search('\u015b\w*', line)
m.group()

NLTK tokenizers allow Unicode strings as input, and correspondingly yield Unicode strings as output.

In [ ]:
word_tokenize(line)

## 3.4   Regular Expressions for Detecting Word Patterns

Many linguistic processing tasks involve pattern matching. For example, we can find words ending with *ed* using `endswith('ed')`. We saw a variety of such "word tests" in Chapter 1. Regular expressions give us a more powerful and flexible method for describing the character patterns we are interested in.

To use regular expressions in Python we need to import the `re` library using: `import re`. We also need a list of words to search; we'll use the Words Corpus again. We will preprocess it to remove any proper names.

In [ ]:
import re
wordlist = [w for w in nltk.corpus.words.words('en') if w.islower()]

### Using Basic Meta-Characters

Let's find words ending with *ed* using the regular expression `«ed$»`. We will use the `re.search(p, s)` function to check whether the pattern `p` can be found somewhere inside the string `s`. We need to specify the characters of interest, and use the dollar sign which has a special behavior in the context of regular expressions in that it matches the end of the word:

In [ ]:
[w for w in wordlist if re.search('ed$', w)]

The `.` **wildcard** symbol matches any single character. Suppose we have room in a crossword puzzle for an 8-letter word with *j* as its third letter and *t* as its sixth letter. In place of each blank cell we use a period:

In [ ]:
[w for w in wordlist if re.search('^..j..t..$', w)]

Finally, the `?` symbol specifies that the previous character is optional. Thus `«^e-?mail$»` will match both `email` and `e-mail`. We could count the total number of occurrences of this word (in either spelling) in a text using `sum(1 for w in text if re.search('^e-?mail$', w))`.

### Ranges and Closures

<img src=images/T9.png width='400' />

*T9: Text on 9 Keys*

The **T9** system is used for entering text on mobile phones. Two or more words that are entered with the same sequence of keystrokes are known as **textonyms**. For example, both *hole* and *golf* are entered by pressing the sequence 4653. What other words could be produced with the same sequence? Here we use the regular expression `«^[ghi][mno][jlk][def]$»`:

In [ ]:
[w for w in wordlist if re.search('^[ghi][mno][jlk][def]$', w)]

The first part of the expression, `«^[ghi]»`, matches the start of a word followed by *g*, *h*, or *i*. The next part of the expression, `«[mno]»`, constrains the second character to be *m*, *n*, or *o*. The third and fourth characters are also constrained. Only four words satisfy all these constraints. Note that the order of characters inside the square brackets is not significant, so we could have written `«^[hig][nom][ljk][fed]$»` and matched the same words.

Let's explore the `+` symbol a bit further. Notice that it can be applied to individual letters, or to bracketed sets of letters:

In [ ]:
chat_words = sorted(set(w for w in nltk.corpus.nps_chat.words()))
[w for w in chat_words if re.search('^m+i+n+e+$', w)]

In [ ]:
[w for w in chat_words if re.search('^[ha]+$', w)]

It should be clear that `+` simply means "one or more instances of the preceding item", which could be an individual character like `m`, a set like `[fed]` or a range like `[d-f]`. Now let's replace `+` with `*`, which means "zero or more instances of the preceding item". The regular expression `«^m*i*n*e*$»` will match everything that we found using `«^m+i+n+e+$»`, but also words where some of the letters don't appear at all, e.g. *me*, *min*, and *mmmmm*. Note that the `+` and `*` symbols are sometimes referred to as **Kleene closures**, or simply **closures**.

The `^` operator has another function when it appears as the first character inside square brackets. For example `«[^aeiouAEIOU]»` matches any character other than a vowel. We can search the NPS Chat Corpus for words that are made up entirely of non-vowel characters using `«^[^aeiouAEIOU]+$»` to find items like these: `:):):)`, `grrr`, `cyb3r` and `zzzzzzzz`. Notice this includes non-alphabetic characters.

Here are some more examples of regular expressions being used to find tokens that match a particular pattern, illustrating the use of some new symbols: `\`, `{}`, `()`, and `|`:

In [ ]:
wsj = sorted(set(nltk.corpus.treebank.words()))
[w for w in wsj if re.search('^[0-9]+\.[0-9]+$', w)]

In [ ]:
[w for w in wsj if re.search('^[A-Z]+\$$', w)]

In [ ]:
[w for w in wsj if re.search('^[0-9]{4}$', w)]

In [ ]:
[w for w in wsj if re.search('^[0-9]+-[a-z]{3,5}$', w)]

In [ ]:
[w for w in wsj if re.search('^[a-z]{5,}-[a-z]{2,3}-[a-z]{,6}$', w)]

In [ ]:
[w for w in wsj if re.search('(ed|ing)$', w)]

You probably worked out that a backslash means that the following character is deprived of its special powers and must literally match a specific character in the word. Thus, while `.` is special, `\.` only matches a period. The braced expressions, like `{3,5}`, specify the number of repeats of the previous item. The pipe character indicates a choice between the material on its left or its right. Parentheses indicate the scope of an operator: they can be used together with the pipe (or disjunction) symbol like this: `«w(i|e|ai|oo)t»`, matching *wit*, *wet*, *wait*, and *woot*. It is instructive to see what happens when you omit the parentheses from the last expression above, and search for `«ed|ing$»`.

The meta-characters we have seen are summarized below.

*Basic Regular Expression Meta-Characters, Including Wildcards, Ranges and Closures*

|Operator| 	Behavior|
|:-|:-|
|`.`| 	Wildcard, matches any character|
|`^abc`| 	Matches some pattern *abc* at the start of a string|
|`abc$`| 	Matches some pattern *abc* at the end of a string|
|`[abc]`| 	Matches one of a set of characters|
|`[A-Z0-9]`| 	Matches one of a range of characters|
|`ed\|ing\|s`| 	Matches one of the specified strings (disjunction)|
|`*`| 	Zero or more of previous item, e.g. `a*`, `[a-z]*` (also known as *Kleene Closure*)|
|`+`| 	One or more of previous item, e.g. `a+`, `[a-z]+`|
|`?`| 	Zero or one of the previous item (i.e. optional), e.g. `a?`, `[a-z]?`|
|`{n}`| 	Exactly *n* repeats where *n* is a non-negative integer|
|`{n,}`| 	At least *n* repeats|
|`{,n}`| 	No more than *n* repeats|
|`{m,n}`| 	At least m and no more than *n* repeats|
|`a(b\|c)+`| 	Parentheses that indicate the scope of the operators|

To the Python interpreter, a regular expression is just like any other string. If the string contains a backslash followed by particular characters, it will interpret these specially. For example `\b` would be interpreted as the backspace character. In general, when using regular expressions containing backslash, we should instruct the interpreter not to look inside the string at all, but simply to pass it directly to the `re` library for processing. We do this by prefixing the string with the letter `r`, to indicate that it is a **raw string**. For example, the raw string `r'\band\b'` contains two `\b` symbols that are interpreted by the `re` library as matching word boundaries instead of backspace characters. If you get into the habit of using `r'...'` for regular expressions — as we will do from now on — you will avoid having to think about these complications.

## 3.5   Useful Applications of Regular Expressions

The above examples all involved searching for words *w* that match some regular expression *regexp* using `re.search(regexp, w)`. Apart from checking if a regular expression matches a word, we can use regular expressions to extract material from words, or to modify words in specific ways.

### Extracting Word Pieces

The `re.findall()` ("find all") method finds all (non-overlapping) matches of the given regular expression. Let's find all the vowels in a word, then count them:

In [ ]:
word = 'supercalifragilisticexpialidocious'
re.findall(r'[aeiou]', word)

In [ ]:
len(re.findall(r'[aeiou]', word))

Let's look for all sequences of two or more vowels in some text, and determine their relative frequency:

In [12]:
wsj = sorted(set(nltk.corpus.treebank.words()))
fd = nltk.FreqDist(vs for word in wsj
                       for vs in re.findall(r'[aeiou]{2,}', word))
fd.most_common(12)

[('io', 549),
 ('ea', 476),
 ('ie', 331),
 ('ou', 329),
 ('ai', 261),
 ('ia', 253),
 ('ee', 217),
 ('oo', 174),
 ('ua', 109),
 ('au', 106),
 ('ue', 105),
 ('ui', 95)]

### Doing More with Word Pieces

Once we can use `re.findall()` to extract material from words, there's interesting things to do with the pieces, like glue them back together or plot them.

It is sometimes noted that English text is highly redundant, and it is still easy to read when word-internal vowels are left out. For example, *declaration* becomes *dclrtn*, and *inalienable* becomes *inlnble*, retaining any initial or final vowel sequences. The regular expression in our next example matches initial vowel sequences, final vowel sequences, and all consonants; everything else is ignored. This three-way disjunction is processed left-to-right, if one of the three parts matches the word, any later parts of the regular expression are ignored. We use `re.findall()` to extract all the matching pieces, and `''.join()` to join them together.

In [13]:
regexp = r'^[AEIOUaeiou]+|[AEIOUaeiou]+$|[^AEIOUaeiou]'
def compress(word):
    pieces = re.findall(regexp, word)
    return ''.join(pieces)

english_udhr = nltk.corpus.udhr.words('English-Latin1')
print(nltk.tokenwrap(compress(w) for w in english_udhr[:75]))

Unvrsl Dclrtn of Hmn Rghts Prmble Whrs rcgntn of the inhrnt dgnty and
of the eql and inlnble rghts of all mmbrs of the hmn fmly is the fndtn
of frdm , jstce and pce in the wrld , Whrs dsrgrd and cntmpt fr hmn
rghts hve rsltd in brbrs acts whch hve outrgd the cnscnce of mnknd ,
and the advnt of a wrld in whch hmn bngs shll enjy frdm of spch and


Next, let's combine regular expressions with conditional frequency distributions. Here we will extract all consonant-vowel sequences from the words of Rotokas, such as *ka* and *si*. Since each of these is a pair, it can be used to initialize a conditional frequency distribution. We then tabulate the frequency of each pair:

In [15]:
rotokas_words = nltk.corpus.toolbox.words('rotokas.dic')
cvs = [cv for w in rotokas_words for cv in re.findall(r'[ptksvr][aeiou]', w)]
cfd = nltk.ConditionalFreqDist(cvs)
cfd.tabulate()

    a   e   i   o   u 
k 418 148  94 420 173 
p  83  31 105  34  51 
r 187  63  84  89  79 
s   0   0 100   2   1 
t  47   8   0 148  37 
v  93  27 105  48  49 


['ka',
 'ka',
 'ka',
 'ka',
 'ka',
 'ro',
 'ka',
 'ka',
 'vi',
 'ko',
 'ka',
 'ka',
 'vo',
 'ka',
 'ka',
 'ko',
 'ka',
 'ka',
 'si',
 'ka',
 'ka',
 'ka',
 'ka',
 'ko',
 'ka',
 'ki',
 'to',
 'ka',
 'ku',
 'pa',
 'to',
 'ka',
 'va',
 'ka',
 'pa',
 'ka',
 'pe',
 'ka',
 'pi',
 'ka',
 'pi',
 'ka',
 'pi',
 'pa',
 'to',
 'ka',
 'pi',
 'si',
 'ka',
 'pi',
 'si',
 'vi',
 'ra',
 'ka',
 'po',
 'ka',
 'po',
 'pa',
 'to',
 'ka',
 'ra',
 'ka',
 're',
 'ka',
 're',
 'ko',
 'ka',
 're',
 'ko',
 'pi',
 'ka',
 're',
 'to',
 're',
 'va',
 'ka',
 'va',
 'ka',
 'va',
 'ka',
 've',
 'ka',
 'ka',
 've',
 'ka',
 'pi',
 'ka',
 've',
 'ka',
 'pi',
 'vi',
 'ra',
 'ka',
 've',
 'ka',
 'vi',
 'ra',
 'ka',
 'ka',
 'ka',
 'ka',
 'ka',
 'ka',
 'ka',
 'ka',
 'ro',
 'ka',
 'ka',
 'ka',
 'ka',
 'so',
 'to',
 'ka',
 'ka',
 'vi',
 'ra',
 'ka',
 'ke',
 'ru',
 'ka',
 'pa',
 'ka',
 'pi',
 'ka',
 'pi',
 'ka',
 'pi',
 'vi',
 'ra',
 'ka',
 're',
 'si',
 'ka',
 're',
 'si',
 'vi',
 'ra',
 'ka',
 'tu',
 'ka',
 'tu',
 'pi',
 'ka',

Examining the rows for *s* and *t*, we see they are in partial "complementary distribution", which is evidence that they are not distinct phonemes in the language. Thus, we could conceivably drop *s* from the Rotokas alphabet and simply have a pronunciation rule that the letter *t* is pronounced *s* when followed by *i*. (Note that the single entry having *su*, namely *kasuari*, 'cassowary' is borrowed from English.)

If we want to be able to inspect the words behind the numbers in the above table, it would be helpful to have an index, allowing us to quickly find the list of words that contains a given consonant-vowel pair, e.g. `cv_index['su']` should give us all words containing *su*. Here's how we can do this:

In [ ]:
cv_word_pairs = [(cv, w) for w in rotokas_words
                         for cv in re.findall(r'[ptksvr][aeiou]', w)]
cv_index = nltk.Index(cv_word_pairs)
cv_index['su']

In [ ]:
cv_index['po']

This program processes each word `w` in turn, and for each one, finds every substring that matches the regular expression `«[ptksvr][aeiou]»`. In the case of the word *kasuari*, it finds *ka*, *su* and *ri*. Therefore, the `cv_word_pairs` list will contain `('ka', 'kasuari')`, `('su', 'kasuari')` and `('ri', 'kasuari')`. One further step, using `nltk.Index()`, converts this into a useful index.

### Finding Word Stems

When we use a web search engine, we usually don't mind (or even notice) if the words in the document differ from our search terms in having different endings. A query for *laptops* finds documents containing *laptop* and vice versa. Indeed, *laptop* and *laptops* are just two forms of the same dictionary word (or lemma). For some language processing tasks we want to ignore word endings, and just deal with word stems.

There are various ways we can pull out the stem of a word. Here's a simple-minded approach which just strips off anything that looks like a suffix:

In [ ]:
def stem(word):
    for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

Although we will ultimately use NLTK's built-in stemmers, it's interesting to see how we can use regular expressions for this task. Our first step is to build up a disjunction of all the suffixes. We need to enclose it in parentheses in order to limit the scope of the disjunction.

In [ ]:
re.findall(r'^.*(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

Here, `re.findall()` just gave us the suffix even though the regular expression matched the entire word. This is because the parentheses have a second function, to select substrings to be extracted. If we want to use the parentheses to specify the scope of the disjunction, but not to select the material to be output, we have to add `?:`, which is just one of many arcane subtleties of regular expressions. Here's the revised version.

In [ ]:
re.findall(r'^.*(?:ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

However, we'd actually like to split the word into stem and suffix. So we should just parenthesize both parts of the regular expression:

In [ ]:
re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

This looks promising, but still has a problem. Let's look at a different word, *processes*:

In [ ]:
re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes')

The regular expression incorrectly found an *-s* suffix instead of an *-es* suffix. This demonstrates another subtlety: the star operator is "greedy" and the `.*` part of the expression tries to consume as much of the input as possible. If we use the "non-greedy" version of the star operator, written `*?`, we get what we want:

In [ ]:
re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes')

This works even when we allow an empty suffix, by making the content of the second parentheses optional:

In [ ]:
re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$', 'language')

This approach still has many problems (can you spot them?) but we will move on to define a function to perform stemming, and apply it to a whole text:

In [ ]:
def stem(word):
    regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem

raw = """DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government.  Supreme executive power derives from
a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = word_tokenize(raw)
[stem(t) for t in tokens]

Notice that our regular expression removed the *s* from *ponds* but also from *is* and *basis*. It produced some non-words like *distribut* and *deriv*, but these are acceptable stems in some applications.

### Searching Tokenized Text

You can use a special kind of regular expression for searching across multiple words in a text (where a text is a list of tokens). For example, `"<a> <man>"` finds all instances of *a man* in the text. The angle brackets are used to mark token boundaries, and any whitespace between the angle brackets is ignored (behaviors that are unique to NLTK's `findall()` method for texts). In the following example, we include `<.*>` which will match any single token, and enclose it in parentheses so only the matched word (e.g. *monied*) and not the matched phrase (e.g. *a monied man*) is produced. The second example finds three-word phrases ending with the word *bro*. The last example finds sequences of three or more words starting with the letter *l*.

In [ ]:
from nltk.corpus import gutenberg, nps_chat
moby = nltk.Text(gutenberg.words('melville-moby_dick.txt'))
moby.findall(r"<a> (<.*>) <man>")

In [ ]:
chat = nltk.Text(nps_chat.words())
chat.findall(r"<.*> <.*> <bro>")

In [ ]:
chat.findall(r"<l.*>{3,}")

It is easy to build search patterns when the linguistic phenomenon we're studying is tied to particular words. In some cases, a little creativity will go a long way. For instance, searching a large text corpus for expressions of the form *x and other ys* allows us to discover hypernyms:

In [ ]:
from nltk.corpus import brown
hobbies_learned = nltk.Text(brown.words(categories=['hobbies', 'learned']))
hobbies_learned.findall(r"<\w*> <and> <other> <\w*s>")

With enough text, this approach would give us a useful store of information about the taxonomy of objects, without the need for any manual labor. However, our search results will usually contain false positives, i.e. cases that we would want to exclude. For example, the result: *demands and other factors* suggests that *demand* is an instance of the type *factor*, but this sentence is actually about wage demands. Nevertheless, we could construct our own ontology of English concepts by manually correcting the output of such searches.

Searching corpora also suffers from the problem of false negatives, i.e. omitting cases that we would want to include. It is risky to conclude that some linguistic phenomenon doesn't exist in a corpus just because we couldn't find any instances of a search pattern. Perhaps we just didn't think carefully enough about suitable patterns.

## 3.6   Normalizing Text

In earlier program examples we have often converted text to lowercase before doing anything with its words, e.g. `set(w.lower() for w in text)`. By using `lower()`, we have **normalized** the text to lowercase so that the distinction between *The* and *the* is ignored. Often we want to go further than this, and strip off any affixes, a task known as stemming. A further step is to make sure that the resulting form is a known word in a dictionary, a task known as lemmatization. We discuss each of these in turn. First, we need to define the data we will use in this section:

In [ ]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government.  Supreme executive power derives from
a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = word_tokenize(raw)

### Stemmers

NLTK includes several off-the-shelf stemmers, and if you ever need a stemmer you should use one of these in preference to crafting your own using regular expressions, since these handle a wide range of irregular cases. The Porter and Lancaster stemmers follow their own rules for stripping affixes. Observe that the Porter stemmer correctly handles the word *lying* (mapping it to *lie*), while the Lancaster stemmer does not.

In [ ]:
porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()
[porter.stem(t) for t in tokens]

In [ ]:
[lancaster.stem(t) for t in tokens]

Stemming is not a well-defined process, and we typically pick the stemmer that best suits the application we have in mind. The Porter Stemmer is a good choice if you are indexing some texts and want to support search using alternative forms of words (illustrated in the code below, which uses *object oriented* programming techniques that are outside the scope of this book, string formatting techniques to be covered in Section 3.9, and the `enumerate()` function to be explained in Chapter 4).

In [ ]:
class IndexedText(object):

    def __init__(self, stemmer, text):
        self._text = text
        self._stemmer = stemmer
        self._index = nltk.Index((self._stem(word), i)
                                 for (i, word) in enumerate(text))

    def concordance(self, word, width=40):
        key = self._stem(word)
        wc = int(width/4)                # words of context
        for i in self._index[key]:
            lcontext = ' '.join(self._text[i-wc:i])
            rcontext = ' '.join(self._text[i:i+wc])
            ldisplay = '{:>{width}}'.format(lcontext[-width:], width=width)
            rdisplay = '{:{width}}'.format(rcontext[:width], width=width)
            print(ldisplay, rdisplay)

    def _stem(self, word):
        return self._stemmer.stem(word).lower()

porter = nltk.PorterStemmer()
grail = nltk.corpus.webtext.words('grail.txt')
text = IndexedText(porter, grail)
text.concordance('lie')

### Lemmatization

The WordNet lemmatizer only removes affixes if the resulting word is in its dictionary. This additional checking process makes the lemmatizer slower than the above stemmers. Notice that it doesn't handle *lying*, but it converts *women* to *woman*.

In [ ]:
wnl = nltk.WordNetLemmatizer()
[wnl.lemmatize(t) for t in tokens]

The WordNet lemmatizer is a good choice if you want to compile the vocabulary of some texts and want a list of valid lemmas (or lexicon headwords).

## 3.7   Regular Expressions for Tokenizing Text

Tokenization is the task of cutting a string into identifiable linguistic units that constitute a piece of language data. Although it is a fundamental task, we have been able to delay it until now because many corpora are already tokenized, and because NLTK includes some tokenizers. Now that you are familiar with regular expressions, you can learn how to use them to tokenize text, and to have much more control over the process.

### Simple Approaches to Tokenization

The very simplest method for tokenizing text is to split on whitespace. Consider the following text from *Alice's Adventures in Wonderland*:

In [9]:
raw = """'When I'M a Duchess,' she said to herself, (not in a very hopeful tone
though), 'I won't have any pepper in my kitchen AT ALL. Soup does very
well without--Maybe it's always pepper that makes people hot-tempered,'..."""

We could split this raw text on whitespace using `raw.split()`. To do the same using a regular expression, it is not enough to match any space characters in the string since this results in tokens that contain a `\n` newline character; instead we need to match any number of spaces, tabs, or newlines:

In [10]:
re.split(r' ', raw)

["'When",
 "I'M",
 'a',
 "Duchess,'",
 'she',
 'said',
 'to',
 'herself,',
 '(not',
 'in',
 'a',
 'very',
 'hopeful',
 'tone\nthough),',
 "'I",
 "won't",
 'have',
 'any',
 'pepper',
 'in',
 'my',
 'kitchen',
 'AT',
 'ALL.',
 'Soup',
 'does',
 'very\nwell',
 'without--Maybe',
 "it's",
 'always',
 'pepper',
 'that',
 'makes',
 'people',
 "hot-tempered,'..."]

In [11]:
re.split(r'[ \t\n]+', raw)

["'When",
 "I'M",
 'a',
 "Duchess,'",
 'she',
 'said',
 'to',
 'herself,',
 '(not',
 'in',
 'a',
 'very',
 'hopeful',
 'tone',
 'though),',
 "'I",
 "won't",
 'have',
 'any',
 'pepper',
 'in',
 'my',
 'kitchen',
 'AT',
 'ALL.',
 'Soup',
 'does',
 'very',
 'well',
 'without--Maybe',
 "it's",
 'always',
 'pepper',
 'that',
 'makes',
 'people',
 "hot-tempered,'..."]

The regular expression `«[ \t\n]+»` matches one or more space, tab (`\t`) or newline (`\n`). Other whitespace characters, such as carriage-return and form-feed should really be included too. Instead, we will use a built-in `re` abbreviation, `\s`, which means any whitespace character. The above statement can be rewritten as `re.split(r'\s+', raw)`.

Splitting on whitespace gives us tokens like `'(not'` and `'herself,'`. An alternative is to use the fact that Python provides us with a character class `\w` for word characters, equivalent to `[a-zA-Z0-9_]`. It also defines the complement of this class `\W`, i.e. all characters other than letters, digits or underscore. We can use `\W` in a simple regular expression to split the input on anything *other* than a word character:

In [ ]:
re.split(r'\W+', raw)

Observe that this gives us empty strings at the start and the end (to understand why, try doing `'xx'.split('x')`). We get the same tokens, but without the empty strings, with `re.findall(r'\w+', raw)`, using a pattern that matches the words instead of the spaces. Now that we're matching the words, we're in a position to extend the regular expression to cover a wider range of cases. The regular expression `«\w+|\S\w*»` will first try to match any sequence of word characters. If no match is found, it will try to match any non-whitespace character (`\S` is the complement of `\s`) followed by further word characters. This means that punctuation is grouped with any following letters (e.g. *'s*) but that sequences of two or more punctuation characters are separated.

In [ ]:
re.findall(r'\w+|\S\w*', raw)

Let's generalize the `\w+` in the above expression to permit word-internal hyphens and apostrophes: `«\w+([-']\w+)*»`. This expression means `\w+` followed by zero or more instances of `[-']\w+`; it would match *hot-tempered* and *it's*. (We need to include `?:` in this expression for reasons discussed earlier.) We'll also add a pattern to match quote characters so these are kept separate from the text they enclose.

In [ ]:
print(re.findall(r"\w+(?:[-']\w+)*|'|[-.(]+|\S\w*", raw))

The above expression also included `«[-.(]+»` which causes the double hyphen, ellipsis, and open parenthesis to be tokenized separately.

Below we list the regular expression character class symbols we have seen in this section, in addition to some other useful symbols.

*Regular Expression Symbols*

|Symbol| 	Function|
|:-|:-|
|`\b`| 	Word boundary (zero width)|
|`\d`| 	Any decimal digit (equivalent to `[0-9]`)|
|`\D`| 	Any non-digit character (equivalent to `[^0-9]`)|
|`\s`| 	Any whitespace character (equivalent to `[ \t\n\r\f\v]`)|
|`\S`| 	Any non-whitespace character (equivalent to `[^ \t\n\r\f\v]`)|
|`\w`| 	Any alphanumeric character (equivalent to `[a-zA-Z0-9_]`)|
|`\W`| 	Any non-alphanumeric character (equivalent to `[^a-zA-Z0-9_]`)|
|`\t`| 	The tab character|
|`\n`| 	The newline character|

### NLTK's Regular Expression Tokenizer

The function `nltk.regexp_tokenize()` is similar to `re.findall()` (as we've been using it for tokenization). However, `nltk.regexp_tokenize()` is more efficient for this task, and avoids the need for special treatment of parentheses. For readability we break up the regular expression over several lines and add a comment about each line. The special `(?x)` "verbose flag" tells Python to strip out the embedded whitespace and comments.

In [ ]:
text = 'That U.S.A. poster-print costs $12.40...'
pattern = r'''(?x)     # set flag to allow verbose regexps
    (?:[A-Z]\.)+       # abbreviations, e.g. U.S.A.
  | \w+(?:-\w+)*       # words with optional internal hyphens
  | \$?\d+(?:\.\d+)?%? # currency and percentages, e.g. $12.40, 82%
  | \.\.\.             # ellipsis
  | [][.,;"'?():-_`]   # these are separate tokens; includes ], [
'''
nltk.regexp_tokenize(text, pattern)

When using the verbose flag, you can no longer use `' '` to match a space character; use `\s` instead. The `regexp_tokenize()` function has an optional `gaps` parameter. When set to `True`, the regular expression specifies the gaps between tokens, as with `re.split()`.

### Further Issues with Tokenization

Tokenization turns out to be a far more difficult task than you might have expected. No single solution works well across-the-board, and we must decide what counts as a token depending on the application domain.

When developing a tokenizer it helps to have access to raw text which has been manually tokenized, in order to compare the output of your tokenizer with high-quality (or "gold-standard") tokens. The NLTK corpus collection includes a sample of Penn Treebank data, including the raw Wall Street Journal text (`nltk.corpus.treebank_raw.raw()`) and the tokenized version (`nltk.corpus.treebank.words()`).

A final issue for tokenization is the presence of contractions, such as *didn't*. If we are analyzing the meaning of a sentence, it would probably be more useful to normalize this form to two separate forms: *did* and *n't* (or *not*). We can do this work with the help of a lookup table.

## 3.8   Segmentation

This section discusses more advanced concepts, which you may prefer to skip on the first time through this chapter.

Tokenization is an instance of a more general problem of **segmentation**. In this section we will look at two other instances of this problem, which use radically different techniques to the ones we have seen so far in this chapter.


### Sentence Segmentation

Manipulating texts at the level of individual words often presupposes the ability to divide a text into individual sentences. As we have seen, some corpora already provide access at the sentence level. In the following example, we compute the average number of words per sentence in the Brown Corpus:

In [ ]:
len(nltk.corpus.brown.words()) / len(nltk.corpus.brown.sents())

In other cases, the text is only available as a stream of characters. Before tokenizing the text into words, we need to segment it into sentences. NLTK facilitates this by including the Punkt sentence segmenter. Here is an example of its use in segmenting the text of a novel:

In [ ]:
text = nltk.corpus.gutenberg.raw('chesterton-thursday.txt')
sents = nltk.sent_tokenize(text)
pprint.pprint(sents[79:89])

Notice that this example is really a single sentence, reporting the speech of Mr Lucian Gregory. However, the quoted speech contains several sentences, and these have been split into individual strings. This is reasonable behavior for most applications.

Sentence segmentation is difficult because period is used to mark abbreviations, and some periods simultaneously mark an abbreviation and terminate a sentence, as often happens with acronyms like *U.S.A.*

For another approach to sentence segmentation, see Chapter 6.

### Word Segmentation

For some writing systems, tokenizing text is made more difficult by the fact that there is no visual representation of word boundaries. For example, in Chinese, the three-character string: 爱国人 (ai4 "love" (verb), guo2 "country", ren2 "person") could be tokenized as 爱国 / 人, "country-loving person" or as 爱 / 国人, "love country-person."

A similar problem arises in the processing of spoken language, where the hearer must segment a continuous speech stream into individual words. A particularly challenging version of this problem arises when we don't know the words in advance. This is the problem faced by a language learner, such as a child hearing utterances from a parent. Consider the following artificial example, where word boundaries have been removed:

(1a) doyouseethekitty

(1b)		seethedoggy

(1c)		doyoulikethekitty

(1d)		likethedoggy

Our first challenge is simply to represent the problem: we need to find a way to separate text content from the segmentation. We can do this by annotating each character with a boolean value to indicate whether or not a word-break appears after the character (an idea that will be used heavily for "chunking" in Chapter 7). Let's assume that the learner is given the utterance breaks, since these often correspond to extended pauses. Here is a possible representation, including the initial and target segmentations:

In [ ]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"

Observe that the segmentation strings consist of zeros and ones. They are one character shorter than the source text, since a text of length *n* can only be broken up in *n-1* places. The `segment()` function demonstrates that we can get back to the original segmented text from the above representation.

In [ ]:
def segment(text, segs):
    words = []
    last = 0
    for i in range(len(segs)):
        if segs[i] == '1':
            words.append(text[last:i+1])
            last = i+1
    words.append(text[last:])
    return words

text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"
segment(text, seg1)

In [ ]:
segment(text, seg2)

Now the segmentation task becomes a search problem: find the bit string that causes the text string to be correctly segmented into words. We assume the learner is acquiring words and storing them in an internal lexicon. Given a suitable lexicon, it is possible to reconstruct the source text as a sequence of lexical items. We can define an **objective function**, a scoring function whose value we will try to optimize, based on the size of the lexicon (number of characters in the words plus an extra delimiter character to mark the end of each word) and the amount of information needed to reconstruct the source text from the lexicon. We illustrate this in the following figure.

<img src= 'images/brent.png' width='600' />

*Calculation of Objective Function: Given a hypothetical segmentation of the source text (on the left), derive a lexicon and a derivation table that permit the source text to be reconstructed, then total up the number of characters used by each lexical item (including a boundary marker) and the number of lexical items used by each derivation, to serve as a score of the quality of the segmentation; smaller values of the score indicate a better segmentation.*

It is a simple matter to implement this objective function:

In [ ]:
def evaluate(text, segs):
    words = segment(text, segs)
    text_size = len(words)
    lexicon_size = sum(len(word) + 1 for word in set(words))
    return text_size + lexicon_size

text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"
seg3 = "0000100100000011001000000110000100010000001100010000001"
segment(text, seg3)

In [ ]:
evaluate(text, seg3)

In [ ]:
evaluate(text, seg2)

In [ ]:
evaluate(text, seg1)

The final step is to search for the pattern of zeros and ones that minimizes this objective function. Notice that the best segmentation includes "words" like *thekitty*, since there's not enough evidence in the data to split this any further.

In [ ]:
from random import randint

def flip(segs, pos):
    return segs[:pos] + str(1-int(segs[pos])) + segs[pos+1:]

def flip_n(segs, n):
    for i in range(n):
        segs = flip(segs, randint(0, len(segs)-1))
    return segs

def anneal(text, segs, iterations, cooling_rate):
    temperature = float(len(segs))
    while temperature > 0.5:
        best_segs, best = segs, evaluate(text, segs)
        for i in range(iterations):
            guess = flip_n(segs, round(temperature))
            score = evaluate(text, guess)
            if score < best:
                best, best_segs = score, guess
        score, segs = best, best_segs
        temperature = temperature / cooling_rate
        print(evaluate(text, segs), segment(text, segs))
    print()
    return segs

text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
anneal(text, seg1, 5000, 1.2)

This is an example of non-deterministic search using **simulated annealing**: begin searching with phrase segmentations only; randomly perturb the zeros and ones proportional to the "temperature"; with each iteration the temperature is lowered and the perturbation of boundaries is reduced.

With enough data, it is possible to automatically segment text into words with a reasonable degree of accuracy. Such methods can be applied to tokenization for writing systems that don't have any visual representation of word boundaries.

## 3.9   Formatting: From Lists to Strings

Often we write a program to report a single data item, such as a particular element in a corpus that meets some complicated criterion, or a single summary statistic such as a word-count or the performance of a tagger. More often, we write a program to produce a structured result; for example, a tabulation of numbers or linguistic forms, or a reformatting of the original data. When the results to be presented are linguistic, textual output is usually the most natural choice. However, when the results are numerical, it may be preferable to produce graphical output. In this section you will learn about a variety of ways to present program output.

### From Lists to Strings

The simplest kind of structured object we use for text processing is lists of words. When we want to output these to a display or a file, we must convert these lists into strings. To do this in Python we use the `join()` method, and specify the string to be used as the "glue".

In [ ]:
silly = ['We', 'called', 'him', 'Tortoise', 'because', 'he', 'taught', 'us', '.']
' '.join(silly)

In [ ]:
';'.join(silly)

In [ ]:
''.join(silly)

So `' '.join(silly)` means: take all the items in `silly` and concatenate them as one big string, using `' '` as a spacer between the items. I.e. `join()` is a method of the string that you want to use as the glue. (Many people find this notation for `join()` counter-intuitive.) The `join()` method only works on a list of strings — what we have been calling a text — a complex type that enjoys some privileges in Python.

### Strings and Formats

We have seen that there are two ways to display the contents of an object:

In [ ]:
word = 'cat'
sentence = """hello
world"""
print(word)

In [ ]:
print(sentence)

In [ ]:
word

In [ ]:
sentence

The `print` command yields Python's attempt to produce the most human-readable form of an object. The second method — naming the variable at a prompt — shows us a string that can be used to recreate this object. It is important to keep in mind that both of these are just strings, displayed for the benefit of you, the user. They do not give us any clue as to the actual internal representation of the object.

There are many other useful ways to display an object as a string of characters. This may be for the benefit of a human reader, or because we want to **export** our data to a particular file format for use in an external program.

Formatted output typically contains a combination of variables and pre-specified strings, e.g. given a frequency distribution `fdist` we could do:

In [ ]:
fdist = nltk.FreqDist(['dog', 'cat', 'dog', 'cat', 'dog', 'snake', 'dog', 'cat'])
for word in sorted(fdist):
    print(word, '->', fdist[word], end='; ')

Print statements that contain alternating variables and constants can be difficult to read and maintain. Another solution is to use **string formatting**.

In [ ]:
for word in sorted(fdist):
   print('{}->{};'.format(word, fdist[word]), end=' ')

To understand what is going on here, let's test out the format string on its own. (By now this will be your usual method of exploring new syntax.)

In [ ]:
'{}->{};'.format ('cat', 3)

The curly brackets `'{}'` mark the presence of a **replacement field**: this acts as a placeholder for the string values of objects that are passed to the `str.format()` method. We can embed occurrences of `'{}'` inside a string, then replace them with strings by calling `format()` with appropriate arguments. A string containing replacement fields is called a **format string**.

Let's unpack the above code further, in order to see this behavior up close:

In [ ]:
'{}->'.format('cat')

In [ ]:
'{}'.format(3)

In [ ]:
'I want a {} right now'.format('coffee')

We can have any number of placeholders, but the `str.format` method must be called with exactly the same number of arguments.

In [ ]:
'{} wants a {} {}'.format ('Lee', 'sandwich', 'for lunch')

In [ ]:
#'{} wants a {} {}'.format ('sandwich', 'for lunch')

Arguments to `format()` are consumed left to right, and any superfluous arguments are simply ignored.

In [ ]:
'{} wants a {}'.format ('Lee', 'sandwich', 'for lunch')

The field name in a format string can start with a number, which refers to a positional argument of `format()`. Something like `'from {} to {}'` is equivalent to `'from {0} to {1}'`, but we can use the numbers to get non-default orders:

In [ ]:
'from {1} to {0}'.format('A', 'B')

We can also provide the values for the placeholders indirectly. Here's an example using a `for` loop:

In [ ]:
template = 'Lee wants a {} right now'
menu = ['sandwich', 'spam fritter', 'pancake']
for snack in menu:
    print(template.format(snack))

### Lining Things Up

So far our format strings generated output of arbitrary width on the page (or screen). We can add padding to obtain output of a given width by inserting into the brackets a colon `':'` followed by an integer. So `{:6}` specifies that we want a string that is padded to width 6. It is right-justified by default for numbers, but we can precede the width specifier with a `'<'` alignment option to make numbers left-justified.

In [ ]:
'{:6}'.format(41)

In [ ]:
'{:<6}' .format(41) 

Strings are left-justified by default, but can be right-justified with the `'>'` alignment option.

In [ ]:
'{:6}'.format('dog')

In [ ]:
'{:>6}'.format('dog') 

Other control characters can be used to specify the sign and precision of floating point numbers; for example `{:.4f}` indicates that four digits should be displayed after the decimal point for a floating point number.

In [ ]:
import math
'{:.4f}'.format(math.pi)

The string formatting is smart enough to know that if you include a `'%'` in your format specification, then you want to represent the value as a percentage; there's no need to multiply by 100.

In [ ]:
count, total = 3205, 9375
"accuracy for {} words: {:.4%}".format(total, count / total)

An important use of formatting strings is for tabulating data. Recall that in Chapter 1 we saw data being tabulated from a conditional frequency distribution. Let's perform the tabulation ourselves, exercising full control of headings and column widths. Note the clear separation between the language processing work, and the tabulation of results.

In [ ]:
def tabulate(cfdist, words, categories):
    print('{:16}'.format('Category'), end=' ')                    # column headings
    for word in words:
        print('{:>6}'.format(word), end=' ')
    print()
    for category in categories:
        print('{:16}'.format(category), end=' ')                  # row heading
        for word in words:                                        # for each word
            print('{:6}'.format(cfdist[category][word]), end=' ') # print table cell
        print()                                                   # end the row

from nltk.corpus import brown
cfd = nltk.ConditionalFreqDist(
          (genre, word)
          for genre in brown.categories()
          for word in brown.words(categories=genre))
genres = ['news', 'religion', 'hobbies', 'science_fiction', 'romance', 'humor']
modals = ['can', 'could', 'may', 'might', 'must', 'will']
tabulate(cfd, modals, genres)

Recall from Section 3.6 that we used a format string `'{:{width}}'` and bound a value to the `width` parameter in `format()`. This allows us to specify the width of a field using a variable:

In [ ]:
'{:{width}}'.format('Monty Python', width=15)

We could use this to automatically customize the column to be just wide enough to accommodate all the words, using `width = max(len(w) for w in words).`

### Writing Results to a File

We have seen how to read text from files (Section 3.1). It is often useful to write output to files as well. The following code opens a file `output.txt` for writing, and saves the program output to the file.

In [ ]:
output_file = open('output.txt', 'w')
words = set(nltk.corpus.genesis.words('english-kjv.txt'))
for word in sorted(words):
    print(word, file=output_file)

When we write non-text data to a file we must convert it to a string first. We can do this conversion using formatting strings, as we saw above. Let's write the total number of words to our file:

In [ ]:
len(words)

In [ ]:
str(len(words))

In [ ]:
print(str(len(words)), file=output_file)
output_file.close()

### Text Wrapping

When the output of our program is text-like, instead of tabular, it will usually be necessary to wrap it so that it can be displayed conveniently. Consider the following output, which overflows its line, and which uses a complicated `print` statement:

In [ ]:
saying = ['After', 'all', 'is', 'said', 'and', 'done', ',',
          'more', 'is', 'said', 'than', 'done', '.']
for word in saying:
    print(word, '(' + str(len(word)) + '),', end=' ')

We can take care of line wrapping with the help of Python's `textwrap` module. For maximum clarity we will separate each step onto its own line:

In [ ]:
from textwrap import fill
pieces = ["{} ({}),".format(word, len(word)) for word in saying]
output = ' '.join(pieces)
wrapped = fill(output)
print(wrapped)

Notice that there is a linebreak between `more` and its following number. If we wanted to avoid this, we could redefine the formatting string so that it contained no spaces, e.g. `'%s_(%d),'`, then instead of printing the value of `wrapped`, we could print `wrapped.replace('_', ' ')`.

## 3.10   Summary

- In this book we view a text as a list of words. A "raw text" is a potentially long string containing words and whitespace formatting, and is how we typically store and visualize a text.
- A string is specified in Python using single or double quotes: `'Monty Python'`, `"Monty Python"`.
- The characters of a string are accessed using indexes, counting from zero: `'Monty Python'[0]` gives the value `M`. The length of a string is found using `len()`.
- Substrings are accessed using slice notation: `'Monty Python'[1:5]` gives the value `onty`. If the start index is omitted, the substring begins at the start of the string; if the end index is omitted, the slice continues to the end of the string.
- Strings can be split into lists: `'Monty Python'.split()` gives `['Monty', 'Python']`. Lists can be joined into strings: `'/'.join(['Monty', 'Python'])` gives `'Monty/Python'`.
- We can read text from a file `input.txt` using `text = open('input.txt').read()`. We can read text from `url` using `text = request.urlopen(url).read().decode('utf8')`. We can iterate over the lines of a text file using `for line in open(f)`.
- We can write text to a file by opening the file for writing `output_file = open('output.txt', 'w')`, then adding content to the file `print("Monty Python", file=output_file)`.
- Texts found on the web may contain unwanted material (such as headers, footers, markup), that need to be removed before we do any linguistic processing.
- Tokenization is the segmentation of a text into basic units — or tokens — such as words and punctuation. Tokenization based on whitespace is inadequate for many applications because it bundles punctuation together with words. NLTK provides an off-the-shelf tokenizer `nltk.word_tokenize()`.
- Lemmatization is a process that maps the various forms of a word (such as *appeared*, *appears*) to the canonical or citation form of the word, also known as the lexeme or lemma (e.g. *appear*).
- Regular expressions are a powerful and flexible method of specifying patterns. Once we have imported the `re` module, we can use `re.findall()` to find all substrings in a string that match a pattern.
- If a regular expression string includes a backslash, you should tell Python not to preprocess the string, by using a raw string with an `r` prefix: `r'regexp'`.
- When backslash is used before certain characters, e.g. `\n`, this takes on a special meaning (newline character); however, when backslash is used before regular expression wildcards and operators, e.g. `\.`, `\|`, `\$`, these characters *lose* their special meaning and are matched literally.
- A string formatting expression `template % arg_tuple` consists of a format string `template` that contains conversion specifiers like `%-6s` and `%0.2d`.

